# PTX and SASS (Nvidia)

## C Kernel to Source Code

In [1]:
import numpy as np
import pyopencl as cl
import pyopencl.array as cla

ctx = cl.create_some_context(answers=["nvi", 1])
queue = cl.CommandQueue(ctx)

In [2]:
prg = cl.Program(ctx, """
__kernel void sum(
    __global const float *a_g, __global const float *b_g, __global float *res_g)
{
  int gid = get_global_id(0);
  res_g[gid] = a_g[gid] + b_g[gid];
}
""").build()

print(prg.binaries[0].decode())

Comments:

* Intel or AT&T style?
* Note: address spaces always explicit
* What is `ctaid.x`? `%ntid.x`?
* How does parameter passing work?
* Is this the lowest-level abstraction?

In [3]:
!mkdir -p tmp

hacked_binary = prg.binaries[0].replace(b".version 6.1", b".version 6.0")

with open("tmp/binary.ptx", "wb") as outf:
    outf.write(hacked_binary)
    
!(cd tmp; ptxas --gpu-name sm_61 --verbose binary.ptx -o binary.o)

In [4]:
!/usr/local/cuda/bin/cuobjdump --dump-sass tmp/binary.o

## Is Division Expensive?

In [13]:
prg = cl.Program(ctx, """
__kernel void sum(
    __global float *a_g, int n)
{
    int gid = get_global_id(0);
    
    // try dividing by n
    
    int row = gid / 117;
    int col = gid % 117;
    
    a_g[row * 128 + col] *= 2;
    // a_g[gid] *= 2;
    
}
""").build()

hacked_binary = prg.binaries[0].replace(b".version 6.1", b".version 6.0")

with open("tmp/binary.ptx", "wb") as outf:
    outf.write(hacked_binary)
    
!(cd tmp; ptxas --gpu-name sm_60 --verbose binary.ptx -o binary.o)
!/usr/local/cuda/bin/cuobjdump --dump-sass tmp/binary.o | cut -c -80

## An Example with Control Flow

In [68]:
prg = cl.Program(ctx, """
__kernel void sum(
    __global const float *a_g, __global const float *b_g, __global float *res_g, int n)
{
    int gsize = get_global_size(0);
    
    for (int i = get_global_id(0); i < n; i += gsize)
      res_g[i] = a_g[i] + b_g[i];
      
    res_g[get_global_id(0)] = 15;
}
""").build()

hacked_binary = prg.binaries[0].replace(b".version 6.1", b".version 6.0")

with open("tmp/binary.ptx", "wb") as outf:
    outf.write(hacked_binary)
    
!(cd tmp; ptxas --gpu-name sm_60 --verbose binary.ptx -o binary.o)
!/usr/local/cuda/bin/cuobjdump --dump-sass tmp/binary.o | cut -c -80

* Spot something that doesn't quite seem to belong?

## From CUDA

Vector add stolen from [ORNL](https://www.olcf.ornl.gov/tutorials/cuda-vector-addition/).

In [34]:
%%writefile tmp/vector-add.cu

#include <stdio.h>
#include <stdlib.h>
#include <math.h>
 
__global__ void vecAdd(double *a, double *b, double *c, int n)
{
    // Get our global thread ID
    int id = blockIdx.x*blockDim.x+threadIdx.x;
 
    // Make sure we do not go out of bounds
    if (id < n)
        c[id] = a[id] + b[id];
}
 
int main( int argc, char* argv[] )
{
    // Size of vectors
    int n = 100000;
 
    double *h_a;
    double *h_b;
    double *h_c;
 
    double *d_a;
    double *d_b;
    double *d_c;
 
    size_t bytes = n*sizeof(double);
 
    h_a = (double*)malloc(bytes);
    h_b = (double*)malloc(bytes);
    h_c = (double*)malloc(bytes);
 
    cudaMalloc(&d_a, bytes);
    cudaMalloc(&d_b, bytes);
    cudaMalloc(&d_c, bytes);
 
    int i;
    for( i = 0; i < n; i++ ) {
        h_a[i] = sin(i)*sin(i);
        h_b[i] = cos(i)*cos(i);
    }
 
    // Copy host vectors to device
    cudaMemcpy( d_a, h_a, bytes, cudaMemcpyHostToDevice);
    cudaMemcpy( d_b, h_b, bytes, cudaMemcpyHostToDevice);
 
    int blockSize, gridSize;
 
    blockSize = 1024;
 
    gridSize = (int)ceil((float)n/blockSize);
 
    vecAdd<<<gridSize, blockSize>>>(d_a, d_b, d_c, n);
 
    cudaMemcpy( h_c, d_c, bytes, cudaMemcpyDeviceToHost );
 
    double sum = 0;
    for(i=0; i<n; i++)
        sum += h_c[i];
    printf("final result: %f\n", sum/n);
 
    cudaFree(d_a);
    cudaFree(d_b);
    cudaFree(d_c);
 
    free(h_a);
    free(h_b);
    free(h_c);
 
    return 0;
}

In [35]:
!(cd tmp; nvcc -c -ccbin g++-7 vector-add.cu)
!/usr/local/cuda/bin/cuobjdump --dump-sass tmp/vector-add.o

* What is `_Z6vecAddPdS_S_i`?

In [36]:
!echo _Z6vecAddPdS_S_i | c++filt

## Inline PTX

In [47]:
prg = cl.Program(ctx, """
__kernel void getlaneid(__global int *d_ptr, int length)
{
    int elemID = get_global_id(0);
    if (elemID < length)
    {
        unsigned int laneid;
        asm("mov.u32 %0, %%laneid;" : "=r"(laneid));
        d_ptr[elemID] = laneid;
    }
}
""").build()

print(prg.binaries[0].decode())

* What do the constraints mean again?
* Spot the inline assembly
* Observe how the `if` is realized
* Observe the realization of `get_global_id()`

In [48]:
a = cla.empty(queue, 5000, np.uint32)
prg.getlaneid(queue, lanes.shape, None, a.data, np.uint32(a.size))

In [50]:
a[:500]